In [17]:
from json import load

In [18]:
with open("../data/example_build.json","r") as build_file:
    build_config = load(build_file)
    
with open("../data/layer_config.json","r") as config_file:
    layer_config = load(config_file)

In [41]:
inputs = []
dataset = None

for _id,config in build_config.items():
    if config['type'] == 'Input':
        inputs.append(_id)
    elif config['type'] == 'Dataset':
        dataset = _id
    
build_config['input_nodes'] = inputs

levels = [ set() for i in range(len(build_config))]
def setLevel(node,config,di=0):
    levels[di].add(node)
    if build_config[node]['connections']['outbound']:
        for next_node in build_config[node]['connections']['outbound']: 
            setLevel(next_node,build_config,di+1)


for inp in inputs:
    setLevel(inp,build_config,0)
    
build_config['levels'] = levels
    
levels = [list(level) for level in levels if len(level)]
levels

TypeError: list indices must be integers or slices, not str

In [42]:
def build_inbound(inbound:list)->str:
    return ( "([" + ", ".join(inbound) + "])" if len(inbound) > 1 else "(" + inbound[0] + ")" ) if len(inbound) else ""

def set_argument(argument,config):
    value = config['value']
    value = ( None if value == 'None' else f"'{value}'" ) if config['type'] == 'text' else value
    return f"    {argument}={value},"
    
def build_arguments(arguments:dict)->str:
    arguments = '\n'.join([ set_argument(arg,cnf) for arg,cnf in arguments.items() ])
    return arguments

def build_input(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    return f"""#<{layer['id']}>
{layer['id']} = layers.Input(
{arguments}
)
#</{layer['id']}>
"""

def build_dense(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    inbound = build_inbound(layer['connections']['inbound'])
    
    return f"""#<{layer['id']}>
{layer['id']} = layers.{layer['type']}(
{arguments}
){inbound}
#</{layer['id']}>
"""

def build_conv2d(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    inbound = build_inbound(layer['connections']['inbound'])
    
    return f"""#<{layer['id']}>
{layer['id']} = layers.{layer['type']}(
{arguments}
){inbound}
#</{layer['id']}>
"""

def build_globalaverage2d(layer,build_config,*args,**kwargs)->str:
    inbound = layer['connections']['inbound']
    inbound = build_inbound(layer['connections']['inbound'])
    
    return f"""#<{layer['id']}>
{layer['id']} = layers.{layer['type']}(){inbound}
#</{layer['id']}>
"""

def build_model(layer,build_config,*args,**kwargs)->str:
    return f"""##<{layer['id']}>
{layer['id']} = keras.Model(
    [ {', '.join(build_config['input_nodes'])}, ],
    [ {', '.join(layer['connections']['inbound'])}, ]
)
#</{layer['id']}>
"""

def build_compile(layer,build_config,*args,**kwargs)->str:
    model, = layer['connections']['inbound']
    return f"""#<{layer['id']}>
{model}.compile(
    optimizer='{layer['arguments']['optmizer']['value']}',
    loss='{layer['arguments']['loss']['value']}'
)
#</{layer['id']}>
"""

def build_train(layer,build_config,*args,**kwargs)->str:
    return ""

build_functions = {
    "Input":build_input,
    "Conv2D":build_conv2d,
    "GlobalAveragePooling2D":build_globalaverage2d,
    "Dense":build_dense,
    "Model":build_model,
    "Compile":build_compile,
    "Train":build_train
}

build = "#<layers>\n\n"

for level in levels:
    for layer in level:
        layer = build_config[layer]
        build += build_functions[layer['type']](layer,build_config) + '\n'

build += '#</layers>'

In [50]:
code = """-*- Code generated by Tensorflow GUI -*-
#<import>
import pandas as pd
import nympy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizer,losses,metrics
#</import>

{dataset}

dataset = Dataset()

{build}
""".format(
    dataset=build_config[dataset]['arguments']['dataset']['value'],
    build=build
)

print (code)

-*- Code generated by Tensorflow GUI -*-
#<import>
import pandas as pd
import nympy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizer,losses,metrics
#</import>

"""
Note : Don't change dataset id.
All the required packages have been imported with their standard namespaces.

tensorflow as tf
keras as keras
pandas as pd
numpy as np

from sklearn.model_selection , train_test_split
"""

#<dataset id=__id__>
class Dataset:
    """
    Dataset will be used in training 

    The dataset object needs to have following attributes

    train_x : np.ndarray -> Training features
    train_y : np.ndarray -> Training labels 
    test_x : np.ndarray -> Testing features
    test_y : np.ndarray -> Testing labels

    validate : bool -> Weather use validation data or not

    batch_size : int -> Batch size
    epochs : int -> Number of epochs
    batches : int -> Number of batches ( Will be calculated automatically )
    """
    train_x = None
    